In [ ]:
! pip install pandas

In [ ]:
# !conda list


### CSV文件处理

In [6]:
import pandas as pd
df = pd.read_csv('data.csv', encoding='gbk')
df.head()

,建言类型,问题,改进意见
0,安全隐患,用户隐私数据保护措施不完善,加强数据加密和权限申请，确保用户个人信息安全。
1,安全隐患,存在安全漏洞容易受到黑客攻击,加强网络安全防护，定期进行漏洞修复和安全测试，确保用户数据的安全性。
2,安全隐患,系统登录认证方式薄弱，易受到非法访问,改进系统登录认证机制，增加多重身份验证，提高账号安全性。
3,安全隐患,缺乏日志监控和异常检测机制,建立日志监控和异常检测系统，及时发现和应对安全威胁。
4,安全隐患,内部员工权限管理混乱，存在安全隐患,加强员工权限管理，实行权限分级制度，防止数据泄露。


In [ ]:
df.rename(columns={'建言类型':'label', '问题':'text', '改进意见':'suggestions'}, inplace=True)
# df.head()
df['label'] = df['label'].apply(lambda x: str(x))
df['text'] = df['text'].apply(lambda x: str(x))
df['suggestions'] = df['suggestions'].apply(lambda x: str(x))
df['text1'] = df['text'] + ',' + df['suggestions']

df.head()


In [8]:
# temp = df['text1']
df.drop(labels=['text'], axis=1, inplace=True)
df.drop(labels=['suggestions'], axis=1, inplace=True)
df.rename(columns={'text1':'text'}, inplace=True)
df.head()

,label,text
0,安全隐患,"用户隐私数据保护措施不完善,加强数据加密和权限申请，确保用户个人信息安全。"
1,安全隐患,"存在安全漏洞容易受到黑客攻击,加强网络安全防护，定期进行漏洞修复和安全测试，确保用户数据的安全性。"
2,安全隐患,"系统登录认证方式薄弱，易受到非法访问,改进系统登录认证机制，增加多重身份验证，提高账号安全性。"
3,安全隐患,"缺乏日志监控和异常检测机制,建立日志监控和异常检测系统，及时发现和应对安全威胁。"
4,安全隐患,"内部员工权限管理混乱，存在安全隐患,加强员工权限管理，实行权限分级制度，防止数据泄露。"


### 划分数据集

In [ ]:
! pip install scikit_learn



In [ ]:
from sklearn.model_selection import train_test_split
label = df.label
df.drop(columns='label', inplace=True)

df_train, df_test, y_train, y_test = train_test_split(df, label, test_size = 0.2, stratify = label, random_state = 42)
df_train['label'] = y_train

In [21]:
# df_train
# df['text']

,text,label
76,"客户服务质量不稳定，需提升服务一致性,制定统一的客户服务标准，确保服务质量一致性和稳定性。",客户服务
65,"客户服务流程复杂，需简化服务流程,简化客户服务流程，提高服务效率，节省客户时间。",客户服务
161,"网站内容更新不及时，影响用户粘性,定期更新网站内容，提供有价值的信息，增加用户粘性。",网络运营
192,"存在安全漏洞容易受到黑客攻击,加强网络安全防护，定期进行漏洞修复和安全测试，确保用户数据的安全性。",安全隐患
54,"客户投诉处理流程繁琐，影响用户满意度,简化投诉处理流程，提高问题解决效率，增强用户体验。",客户服务
...,...,...
157,"网站信息分类不清晰，导航不便捷,优化网站信息分类，优化导航结构，提升用户浏览体验。",网络运营
36,"系统漏洞修复不及时，易遭受黑客攻击,建立及时漏洞修复流程，确保系统安全稳定。",安全隐患
89,"客户服务响应速度慢，需优化服务流程,优化客户服务流程，提高服务响应速度，增强客户体验。",客户服务
147,"市场调研数据分析不够深入，需要更准确的数据支持,加强市场调研团队能力，深入分析市场数据。",市场运营


In [ ]:
# 拆分训练集和验证集
label = df_train.label
df_train.drop(columns='label', inplace=True)

df_train, df_val, y_train, y_val = train_test_split(
    df_train, label, test_size=0.05, stratify=label, random_state=42
)
df_train['label'] = y_train
df_val['label'] = y_val

In [ ]:
# !pip install datasets

In [ ]:
# 更改为transformers数据集
from datasets import Dataset, DatasetDict
data = DatasetDict()
data['train'] = Dataset.from_pandas(df_train)
data['validation'] = Dataset.from_pandas(df_val)
data['test'] = Dataset.from_pandas(df_test)


In [ ]:
# !pip install accelerate -U


In [ ]:
# 对文本进行编码
from transformers import AutoTokenizer
model_pretrained = 'bert-base-chinese'
tokenizer = AutoTokenizer.from_pretrained(model_pretrained)

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

encoded = data.map(tokenize, batched=True, batch_size=None)

### 预训练模型微调

In [23]:
! pip install tqdm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     -------------------------------------- 77.1/77.1 kB 857.1 kB/s eta 0:00:00


In [ ]:
import torch
# # 使用m1
# device = torch.device('mps' if torch.has_mps else 'cpu')
# 使用N卡
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#实例化模型
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_pretrained, num_labels=4
).to(device)

# 设置训练参数

from transformers import TrainingArguments
batch_size = 4
logging_steps = len(encoded['train']) // batch_size
model_name = f'{model_pretrained}-finetuned-suggestions'

training_args = TrainingArguments(
    report_to='none',
    output_dir=model_name,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=.01,
    evaluation_strategy='epoch',
    disable_tqdm=False,
    logging_steps=logging_steps,
    log_level='error',
)

# 训练模型和评估
from transformers import Trainer
from sklearn import accuracy_score, f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.prediction.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {'accuracy':'acc', 'f1':f1}

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=encoded['train'],
    eval_dataset=encoded['validation'],
    tokenizer=tokenizer,
)
trainer.train()
trainer.save_model()
